# Домашнее задание № 5 

В данном задании требуется реализовать некоторые из метрик, рассмотренные на лекции.

Все функции, кроме ```compute_gain```, в качестве первых двух аргументов принимают на вход тензоры ```ys_true``` и ```ys_pred```. Это вещественные тензоры ```pytorch``` размерности ```n```, задающие целевые отметки релевантности и предсказанные значения соответственно. 

Для генерации примеров входных данных можете использовать ```torch.rand(n)```, если не указана специфика исходных тензоров. 

Считается, что ```ys_pred``` содержит уникальные значения без повторений.

In [4]:
import math
from math import log2

import torch
from torch import Tensor, sort

## Swapped Pairs

```num_swapped_pairs``` — функция для расчёта количества неправильно упорядоченных пар (корректное упорядочивание — от наибольшего значения в ```ys_true``` к наименьшему) или переставленных пар.

In [5]:
def num_swapped_pairs(ys_true: Tensor, ys_pred: Tensor) -> int:
    _, idxs = torch.sort(ys_pred, descending = True)
    true_sorted = ys_true[idxs]
    
    count = 0
    for i in range(len(ys_true)):
        count += torch.sum(true_sorted[i:] > true_sorted[i])
    
    return int(count)

In [6]:
# не изменять
ys_true = torch.tensor([2, 1, 0, 1, 2])
ys_pred = torch.tensor([0.1, 0.3, 0.2, 0.13, 0.12])

res = num_swapped_pairs(ys_true, ys_pred)
print(res)  # 7

7


## Gain

```compute_gain``` — вспомогательная функция для расчёта DCG и NDCG, рассчитывающая показатель Gain. Принимает на вход дополнительный аргумент — указание схемы начисления Gain (```gain_scheme```).

Необходимо реализовать метод при:
- ```gain_scheme="const"``` - постоянное начисление Gain
- ```gain_scheme="exp2"``` - рассчитываемый по формуле $(2^r −1)$, где $r$ — реальная релевантность документа некоторому запросу.

In [7]:
def compute_gain(y_value: float, gain_scheme: str) -> float:
    if gain_scheme == 'const':
        return float(y_value)

    elif gain_scheme == 'exp2':
        return float(2 ** y_value - 1)
    
    return float("inf")

In [8]:
# не изменять
value = 5

res = compute_gain(value, 'exp2')
print(res)  # 31

res = compute_gain(value, 'const')
print(res)  # 5

31.0
5.0


## DCG

```dcg``` и ```ndcg``` — функции расчёта DCG и NDCG. Принимают на вход дополнительный параметр ```gain_scheme```, аналогичный таковому в функции ```compute_gain```.

In [9]:
def dcg(ys_true: Tensor, ys_pred: Tensor, gain_scheme: str) -> float:
    idx = torch.argsort(ys_pred, descending=True)
    true_sorted = ys_true[idx].to(torch.float64)

    steps = torch.arange(2, ys_pred.size()[0] + 2, dtype=torch.float64)
    steps = torch.log2(steps)

    gains = true_sorted.apply_(lambda x: compute_gain(x, gain_scheme))
    
    return float(torch.sum(gains / steps))
    

def ndcg(ys_true: Tensor, ys_pred: Tensor, gain_scheme: str = 'const') -> float:
    dcg_score = dcg(ys_true, ys_pred, gain_scheme)

    true_sorted, _ = torch.sort(ys_true, descending=True)
    ideal_dcg = dcg(true_sorted, true_sorted, gain_scheme)
    
    return float(dcg_score / ideal_dcg)

In [10]:
# не изменять
ys_true = torch.tensor([2, 2, 4, 1, 2, 0])
ys_pred = torch.tensor([0.1, 0.3, 0.2, 0.14, 0.12, 0.6])

res = dcg(ys_true, ys_pred, gain_scheme='exp2')
res_n = ndcg(ys_true, ys_pred, gain_scheme='exp2')
print(res)  # 12.052645801815459
print(res_n)  # 0.6004804162123548

12.052645801815459
0.6004804162123549


## Precission@k


```precission_at_k``` — функция расчёта точности в топ-k позиций для бинарной разметки (в ```ys_true``` содержатся только нули и единицы). Если среди лейблов нет ни одного релевантного документа (единицы), то необходимо вернуть -1. 

Функция принимает на вход параметр k, указывающий на то, по какому количеству объектов необходимо произвести расчёт метрики. Учтите, что k может быть больше количества элементов во входных тензорах.

In [11]:
def precission_at_k(ys_true: Tensor, ys_pred: Tensor, k: int) -> float:
    k = min(len(ys_pred), k)
    sort_idxs = torch.argsort(ys_pred, descending=True)
    pred_sorted = ys_true[sort_idxs]
    true_positive = torch.sum(pred_sorted[:k] == 1)

    return float(true_positive / k) if true_positive > 0 else -1

In [12]:
# не изменять
ys_true = torch.tensor([0, 0, 1, 1, 0, 1])
ys_pred = torch.tensor([0.1, 0.3, 0.2, 0.14, 0.12, 0.6])

res = precission_at_k(ys_true, ys_pred, k=3)
print(res)  # 0.6666666865348816

0.6666666865348816


## Average Precision

```average_precision``` — функция расчёта AP для бинарной разметки (в ```ys_true``` содержатся только нули и единицы). Если среди лейблов нет ни одного релевантного документа (единицы), то необходимо вернуть -1.

In [13]:
def average_precision(ys_true: Tensor, ys_pred: Tensor) -> float:
    AP = 0
    
    for k in range(1, len(ys_true)+1):
        indx = min(len(ys_pred), k)
        _, sort_idxs = torch.sort(ys_pred, descending=True)
        pred_sorted = ys_true[sort_idxs]
        true_positive = torch.sum(pred_sorted[:indx] == 1)
        AP_k = float(true_positive / torch.sum(ys_true))

        indx = min(len(ys_pred), k-1)
        _, sort_idxs = torch.sort(ys_pred, descending=True)
        pred_sorted = ys_true[sort_idxs]
        true_positive = torch.sum(pred_sorted[:indx] == 1)
        AP_k -= float(true_positive / torch.sum(ys_true))

        AP += AP_k * precission_at_k(ys_true, ys_pred, k)
    return float(AP)

In [14]:
# не изменять
ys_true = torch.tensor([1, 0, 1, 1, 0, 1, 0, 0])
ys_pred = torch.tensor([0.9, 0.8, 0.7, 0.6, 0.4, 0.3, 0.2, 0.1])

res = average_precision(ys_true, ys_pred)
print(res)  # 0.7708333333333333

0.7708333432674408


## reciprocal_rank

```reciprocal_rank``` — функция для расчёта MRR (без усреднения). В ```ys_true``` могут содержаться только нули и максимум одна единица. 

In [15]:
def reciprocal_rank(ys_true: Tensor, ys_pred: Tensor) -> float:
    _, sort_idxs = torch.sort(ys_pred, descending=True)
    pred_sorted = ys_true[sort_idxs]
    rank = torch.argmax(pred_sorted)
    return float(1 / (rank+1))

In [16]:
# не изменять
ys_true = torch.tensor([0, 0, 0, 1, 0, 0])
ys_pred = torch.tensor([0.1, 0.3, 0.2, 0.14, 0.12, 0.6])

res = reciprocal_rank(ys_true, ys_pred)
print(res)  # 0.25

0.25


## p_found

```p_found``` — функция расчёта P-found от Яндекса, принимающая на вход дополнительный параметр ```p_break``` — вероятность прекращения просмотра списка документов в выдаче. Базовая вероятность просмотреть первый документ в выдаче ($pLook[0]$) равняется единице. ```ys_true``` нормированы от 0 до 1 (вероятность удовлетворения запроса пользователя).

In [18]:
def p_found(ys_true: Tensor, ys_pred: Tensor, p_break: float = 0.15 ) -> float:
    pred_sorted, sorted_idxs = torch.sort(ys_pred, descending=False)
    true_sorted = ys_true[sorted_idxs]
    
    p_look = [1]
    for i in range(1, len(ys_true)):
        p_look += [
            p_look[i-1] * (1-true_sorted[i-1]) * (1-p_break)
        ]
    
    p_found = 0.0
    
    for i in range(len(ys_true)):
        p_found += p_look[i] * true_sorted[i]
    
    return float(p_found)

In [19]:
# не изменять
ys_true = torch.tensor([0, 0, 0, 1, 0, 1])
ys_pred = torch.tensor([0.91, 0.72, 0.12, 0.24, 0.15, 0.6])

res = p_found(ys_true, ys_pred, 0.12)
print(res)  # 0.7744

0.774399995803833
